# **Factor Models**


## **Part I: CAPM and Basic Factor Models**

**Objective**: Use Python to test the Capital Asset Pricing Model (CAPM) and the <br>
Arbitrage Pricing Theory (APT) using the Fama–MacBeth regression approach.

### **1. Import and clean the return data**
Read the monthly return data, ensure returns are numeric, and remove invalid entries

In [18]:
import statsmodels.api as sm
import pandas as pd
import numpy as np

### **2. Importing Data**

Load the dataset named sample factors 2000 2024.csv using the pandas library

In [19]:
df = pd.read_csv("data/sample_factors_2000_2024.csv")
df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")
df = df.set_index(['permno','date'])

display(df)

ret_excess  beta_capm_lag   size_lag    bm_lag    mom_lag  \
permno date                                                                    
10001  2000-01-01     -4.8218       0.071436   3.036154  0.842113  -6.622040   
       2000-02-01      1.1085       0.006209   2.991034  0.908207 -10.741689   
       2000-03-01     -2.0458       0.017613   3.006301  0.894447  -0.305305   
       2000-04-01      0.7119       0.038809   2.981228  0.917157  -3.149204   
       2000-05-01     -2.8166       0.032691   2.992878  0.906534  -3.394318   
...                       ...            ...        ...       ...        ...   
93436  2023-08-01     -3.9462       1.655677  13.651604  0.073885 -10.001587   
       2023-09-01     -3.4756       1.744596  13.616016  0.076562  -6.360260   
       2023-10-01    -20.2046       1.794596  13.586662  0.078842  -5.666240   
       2023-11-01     19.0979       1.966896  13.366806  0.098229 -11.734035   
       2023-12-01      3.0688       2.109528  13.545269  0.082174  23.307803   

                      illiq_lag  idio_vol_ff3_lag  max_ret_lag  ret_lag  
permno date                                                              
10001  2000-01-01  2.295151e+00         41.681771     0.104478  -0.4188  
       2000-02-01  2.283478e+00         41.296297     0.104478  -4.4118  
       2000-03-01  2.032289e+00         39.297158     0.104478   1.5385  
       2000-04-01  2.145244e+00         39.883042     0.104478  -1.5758  
       2000-05-01  2.465580e+00         40.633927     0.104478   1.1719  
...                         ...               ...          ...      ...  
93436  2023-08-01  1.164666e-06         49.356502     0.110002   2.1622  
       2023-09-01  1.161371e-06         49.464644     0.110002  -3.4962  
       2023-10-01  1.113744e-06         50.751133     0.110002  -3.0456  
       2023-11-01  1.049690e-06         49.613916     0.110002 -19.7346  
       2023-12-01  9.568811e-07         48.760425     0.110002  19.5379  

[1001733 rows x 9 columns]

The dataframe has the following columns:

`ret_excess  beta_capm_lag  size_lag  bm_lag  mom_lag`  <br>
`illiq_lag  idio_vol_ff3_lag  max_ret_lag  ret_lag ` <br>

where `beta_capm_lag` represents the estimated CAPM beta for each stock, lagged by one period.


### **3. Summary Statistics**

Compute and report summary statistics for `beta_capm_lag`:  

Mean, Standard Deviation, Skewness, Kurtosis, Minimum, 5\%, 25\%, Median, 75\%, 95\%, Maximum, and sample size (n).


In [20]:
desc = df["beta_capm_lag"].describe(percentiles=[0.05, 0.25, 0.5, 0.75, 0.95])

summary = pd.Series({
    "Mean": desc["mean"], "Std. Dev.": desc["std"],
    "Skewness": df["beta_capm_lag"].skew(),
    "Kurtosis": df["beta_capm_lag"].kurt(),  
    "Min": desc["min"], "5%": desc["5%"],
    "25%": desc["25%"], "Med": desc["50%"],
    "75%": desc["75%"], "95%": desc["95%"],
    "Max": desc["max"], "n": int(desc["count"]),
}).to_frame("beta_capm_lag").round(4)

summary.T

,Mean,Std. Dev.,Skewness,Kurtosis,Min,5%,25%,Med,75%,95%,Max,n
beta_capm_lag,0.9838,0.587,0.4181,1.5477,-9.8011,0.086,0.5815,0.9664,1.3366,1.9836,11.1627,1001733.0


### **4. Average Cross-Sectional Correlation**

Calculate the average pairwise correlation between all variables across stocks for each time period, <br>
then average over time. *Hint:* Use `df.corr()` within each date group and then average the off-diagonal elements.


In [21]:
corr_by_period = (
    df.groupby("date")
        # df.corr() within each group
      .apply(lambda g: g["ret_excess"].corr(g["beta_capm_lag"]))
      .rename("corr_ret_beta")
).to_frame()

display(corr_by_period)

,corr_ret_beta
date,
2000-01-01,-0.021886
2000-02-01,0.111170
2000-03-01,-0.048830
2000-04-01,-0.217119
2000-05-01,-0.289479
...,...
2023-08-01,-0.202901
2023-09-01,-0.189022
2023-10-01,-0.234168


In [22]:
print(f"Average cross-sectional correlation: {corr_by_period["corr_ret_beta"].mean():.4f}")

Average cross-sectional correlation: -0.0162


### **5. Testing the CAPM (Fama–MacBeth Regression)**

Run the Fama–MacBeth regression to test the CAPM hypothesis using the code template. Report:
- Intercept (alpha)
- Coefficient(s) on beta(s)
- Adjusted $R^2$
- t-statistics and p-values

**Note**: using `df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d").dt.to_period('M')`  <br>
is not working with `linearmodels.panel.FamaMacBeth` so we need full date: <br>
`df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")`


In [23]:
from linearmodels.panel import FamaMacBeth
import statsmodels.api as sm

X_var = ["beta_capm_lag"]
data = df[["ret_excess"] + X_var].dropna()
exog = sm.add_constant(data[X_var])

mod = FamaMacBeth(data["ret_excess"], exog)
fe_res = mod.fit(cov_type='kernel', kernel='bartlett', bandwidth=12)
display(fe_res.summary)

Dep. Variable:,ret_excess,R-squared:,9.012e-05
Estimator:,FamaMacBeth,R-squared (Between):,-0.0559
No. Observations:,1001733,R-squared (Within):,0.0001
Date:,"Mon, Oct 20 2025",R-squared (Overall):,9.012e-05
Time:,13:05:40,Log-likelihood,-4.212e+06
Cov. Estimator:,Fama-MacBeth Kernel Cov,,
,,F-statistic:,90.280
Entities:,11588,P-value,0.0000
Avg Obs:,86.446,Distribution:,"F(1,1001731)"
Min Obs:,1.0000,,
Max Obs:,288.00,F-statistic (robust):,0.1859


In [24]:
print(f"Alpha (Intercept):       {fe_res.params['const']:.4f}")
print(f"Alpha t-statistic:       {fe_res.tstats['const']:.3f}")
print(f"Beta Coefficient:        {fe_res.params['beta_capm_lag']:.4f}")
print(f"Beta t-statistic:        {fe_res.tstats['beta_capm_lag']:.3f}")
print(f"Alpha p-value:           {fe_res.pvalues['const']:.4f}")
print(f"Beta p-value:            {fe_res.pvalues['beta_capm_lag']:.4f}")
print(f"Adjusted R²:             {fe_res.rsquared:.4f}")

Alpha (Intercept):       0.7800
Alpha t-statistic:       2.573
Beta Coefficient:        -0.1152
Beta t-statistic:        -0.431
Alpha p-value:           0.0101
Beta p-value:            0.6664
Adjusted R²:             0.0001


### **6. Year-by-Year CAPM Analysis (2010–2020)**

Estimate and report the Fama–MacBeth regression results for each year separately.  <br>
Provide coefficients, t-statistics, and $p$-values for each year.


In [25]:
results = []

# Run Fama MacBeth regression for each year
for year in range(2010,2021):
    
    g = df.loc[(df.index.get_level_values("date").year >= year-10) &
               (df.index.get_level_values("date").year <= year)]
    data = g[["ret_excess", "beta_capm_lag"]].dropna()
    if data.empty:
        continue
    exog = sm.add_constant(data[["beta_capm_lag"]])
    mod = FamaMacBeth(data["ret_excess"], exog)
    res = mod.fit(cov_type="kernel", kernel="bartlett", bandwidth=12)

    results.append({
        "Year": year,
        "Alpha": res.params["const"],
        "Beta": res.params["beta_capm_lag"],
        "t_Alpha": res.tstats["const"],
        "t_Beta": res.tstats["beta_capm_lag"],
        "p_Alpha": res.pvalues["const"],
        "p_Beta": res.pvalues["beta_capm_lag"],
        "Adj_R2": res.rsquared
    })

# Combine and display results
results_df = pd.DataFrame(results).set_index("Year").round(4)

results_df


,Alpha,Beta,t_Alpha,t_Beta,p_Alpha,p_Beta,Adj_R2
Year,,,,,,,
2010,0.7456,-0.2551,1.2762,-0.5236,0.2019,0.6005,0.0004
2011,0.6440,-0.0504,1.1030,-0.1305,0.2700,0.8962,0.0001
2012,0.6016,0.0247,1.0641,0.0666,0.2873,0.9469,-0.0000
2013,0.8099,0.2640,1.3720,0.9308,0.1701,0.3519,-0.0001
2014,0.5831,0.1486,1.0967,0.5377,0.2728,0.5908,0.0001
2015,0.4172,0.1406,0.8201,0.5136,0.4121,0.6075,0.0001
2016,0.4412,0.2223,0.8638,0.8000,0.3877,0.4237,0.0003
2017,0.4603,0.2270,0.8931,0.8226,0.3718,0.4107,0.0003
2018,0.4692,0.1699,0.9786,0.5959,0.3278,0.5512,0.0003


### **7. Expected Relationship (CAPM)**

Explain the theoretical prediction: Higher beta $\Rightarrow$ higher expected excess return.  
Test whether this relationship holds empirically.

### **8. Testing the APT (Carhart 4-Factor Model)**

Load the dataset `sample_beta_wrds.csv` with the following columns:

`PERMNO  DATE  n  RET  alpha  b_mkt  b_smb  b_hml  b_umd ` <br>
`ivol  tvol  R2  exret  TICKER`

**Variable Description:**
- `b_mkt`, `b_smb`, `b_hml`, `b_umd`: estimated betas for market, size, value, and momentum factors  
- `exret`: excess return  
- `alpha`: regression intercept  
- `ivol`, `tvol`: idiosyncratic and total volatility  



In [26]:
df = pd.read_csv("data/sample_beta_wrds.csv")
df["DATE"] = pd.to_datetime(df["DATE"], format="%Y-%m-%d")
df = df.set_index(['PERMNO','DATE'])
df["exret"] = pd.to_numeric(df["exret"].astype(str).str.rstrip("%"), errors="coerce")

display(df)

n        RET   alpha   b_mkt   b_smb   b_hml   b_umd  \
PERMNO DATE                                                                 
10001  2003-02-28  205    3.5545%  0.0058  0.0980 -0.0104  0.0747  0.0184   
       2002-01-31  192   -1.3100%  0.0071  0.0627 -0.0229  0.0564  0.0481   
       2003-10-31  213  -13.0435%  0.0059  0.0835  0.0781  0.0991 -0.0693   
       1996-10-31  129   -2.8571%  0.0097  0.0170 -0.1888 -0.0870 -0.0510   
       2003-08-29  211    8.6601%  0.0059  0.1161  0.1008  0.1483 -0.0541   
...                ...        ...     ...     ...     ...     ...     ...   
93436  2024-09-30  171   22.1942%  0.0293  1.2909  1.1748 -1.2759 -0.4539   
       2024-07-31  169   17.2781%  0.0288  1.3023  1.1526 -1.2508 -0.4236   
       2024-06-28  168   11.1186%  0.0279  1.3165  1.0761 -1.2834 -0.4288   
       2024-08-30  170   -7.7390%  0.0285  1.2908  1.1775 -1.2535 -0.4419   
       2024-12-31  174   17.0008%  0.0312  1.2987  1.2258 -1.3021 -0.4477   

                       ivol      tvol        R2    exret TICKER  
PERMNO DATE                                                      
10001  2003-02-28   5.1703%   5.1864%   0.6222%   3.7389   EWST  
       2002-01-31   5.0702%   5.0813%   0.4358%  -1.7334   EWST  
       2003-10-31   7.4294%   7.4501%   0.5566% -14.1760   EWST  
       1996-10-31   5.6373%   5.6550%   0.6238%  -3.8222   EWST  
       2003-08-29   7.3935%   7.4221%   0.7684%   7.8623   EWST  
...                     ...       ...       ...      ...    ...  
93436  2024-09-30  16.2011%  18.3000%  21.6233%  15.9809   TSLA  
       2024-07-31  16.2532%  18.3338%  21.4086%  14.0991   TSLA  
       2024-06-28  16.2799%  18.3631%  21.4023%   6.3713   TSLA  
       2024-08-30  16.2179%  18.3049%  21.5029%  -5.6184   TSLA  
       2024-12-31  16.1968%  18.3530%  22.1168%  20.3059   TSLA  

[1804478 rows x 12 columns]

### **9. Lagging Betas**

Lag each beta variable by one period before using it in regressions.  

*Hint:* Use

``
df[["b_mkt","b_smb","b_hml","b_umd"]] = df.groupby("PERMNO")[["b_mkt","b_smb","b_hml","b_umd"]].shift(1)
``


In [27]:
df[["b_mkt","b_smb","b_hml","b_umd"]] = df.groupby("PERMNO")[["b_mkt","b_smb","b_hml","b_umd"]].shift(1)

### **10. Summary Statistics for Factor Betas**

Report Mean, SD, Skewness, Kurtosis, Min, 5\%, 25\%, Median, 75\%, 95\%, Max, and n for each of:

`b_mkt`, `b_smb`, `b_hml`, `b_umd`.


In [28]:
cols = ["b_mkt","b_smb","b_hml","b_umd"]

desc = df[cols].describe(percentiles=[0.05, 0.25, 0.5, 0.75, 0.95])

summary = pd.concat({
    "Mean": desc.loc["mean"], "Std. Dev.": desc.loc["std"],
    "Skewness": df[cols].skew(), "Kurtosis": df[cols].kurt(),    
    "Min":  desc.loc["min"], "5%": desc.loc["5%"],
    "25%": desc.loc["25%"], "Med": desc.loc["50%"],
    "75%": desc.loc["75%"], "95%": desc.loc["95%"],
    "Max": desc.loc["max"], "n": desc.loc["count"].astype(int)
}, axis=1).round(4)

summary

,Mean,Std. Dev.,Skewness,Kurtosis,Min,5%,25%,Med,75%,95%,Max,n
b_mkt,0.9196,0.4540,-0.2987,5.3540,-6.7075,0.1978,0.6448,0.9271,1.1845,1.6341,5.3221,1789982
b_smb,0.6787,0.7620,3.0862,73.0919,-6.6741,-0.2511,0.1395,0.5497,1.0677,2.0643,32.1916,1789982
b_hml,0.2399,0.6422,0.2661,20.7809,-6.1277,-0.8427,-0.0802,0.2720,0.6056,1.1806,20.5179,1789982
b_umd,-0.1011,0.3373,-0.9175,12.0820,-6.4974,-0.6449,-0.2479,-0.0796,0.0719,0.3700,5.4058,1789982


### **11. Average Cross-Sectional Correlation (APT Variables)**


Compute and discuss the average pairwise correlations among all four lagged beta variables.

In [29]:
cols = ["b_mkt", "b_smb", "b_hml", "b_umd"]

# Compute correlation matrix for each period, then average elementwise
corr_matrices = df.groupby("DATE")[cols].corr()

# Average over time
avg_corr_matrix = corr_matrices.groupby(level=1).mean().round(4)

In [30]:
avg_corr_matrix

,b_mkt,b_smb,b_hml,b_umd
b_hml,0.0745,0.0647,1.0000,0.0274
b_mkt,1.0000,0.1671,0.0745,-0.0889
b_smb,0.1671,1.0000,0.0647,-0.0413
b_umd,-0.0889,-0.0413,0.0274,1.0000


### **12. Fama–MacBeth Regression: Carhart 4-Factor Model**


Run a Fama–MacBeth regression with `exret` as the dependent variable and the lagged betas as regressors.

In [31]:
X_var = ["b_mkt","b_smb","b_hml","b_umd"]
data = df[["exret"] + X_var].dropna()
exog = sm.add_constant(data[X_var])

mod = FamaMacBeth(data["exret"], exog)
fe_res = mod.fit(cov_type='kernel', kernel='bartlett', bandwidth=12)
display(fe_res.summary)


Dep. Variable:,exret,R-squared:,0.0003
Estimator:,FamaMacBeth,R-squared (Between):,-0.0039
No. Observations:,1763606,R-squared (Within):,0.0001
Date:,"Mon, Oct 20 2025",R-squared (Overall):,0.0003
Time:,13:05:51,Log-likelihood,-7.071e+06
Cov. Estimator:,Fama-MacBeth Kernel Cov,,
,,F-statistic:,127.77
Entities:,12572,P-value,0.0000
Avg Obs:,140.28,Distribution:,"F(4,1763601)"
Min Obs:,1.0000,,
Max Obs:,731.00,F-statistic (robust):,11.593


### **13. Reporting Results**


Provide a table summarizing your results:


| Variable | Coefficient | t-Statistic | p-Value | Significance |
|:---------|-----------:|------------:|--------:|:------------:|
| Intercept | 0.0021 | 1.95 | 0.051 | * |
| b\_mkt    | 0.045  | 3.25 | 0.002 | ** |
| b\_smb    | 0.021  | 1.45 | 0.15  |    |
| b\_hml    | 0.018  | 2.10 | 0.04  | *  |
| b\_umd    | 0.030  | 3.90 | 0.001 | *** |

In [32]:
res_table = pd.DataFrame({
    "Variable": fe_res.params.index,
    "Coefficient": fe_res.params.values,
    "t-Statistic": fe_res.tstats.values,
    "p-Value": fe_res.pvalues.values
})

# Add significance stars
def stars(p):
    if p < 0.01: return "***"
    elif p < 0.05: return "**"
    elif p < 0.1: return "*"
    else: return ""
res_table["Significance"] = res_table["p-Value"].apply(stars)

# Format table nicely
res_table = res_table.round({"Coefficient": 4, "t-Statistic": 2, "p-Value": 3})

display(res_table)

,Variable,Coefficient,t-Statistic,p-Value,Significance
0,const,0.4305,4.26,0.000,***
1,b_mkt,-0.3891,-3.56,0.000,***
2,b_smb,0.0842,1.09,0.277,
3,b_hml,-0.1367,-1.58,0.115,
4,b_umd,-0.6322,-4.58,0.000,***


### **14. Interpretation**

Discuss:
- The sign and size of each coefficient (risk premium interpretation)
- Statistical significance
- Whether results align with theory (e.g., momentum premium positive)

### **15. Year-by-Year Carhart 4-Factor Tests (2010–2024)**


Estimate the Carhart 4-Factor model each year.  
For each year, report coefficients and *p*-values for each beta.  
Plot the time series of estimated factor risk premia.



In [33]:
X_var = ["b_mkt", "b_smb", "b_hml", "b_umd"]
results = []

def stars(p):
    return "***" if p < 0.01 else "**" if p < 0.05 else "*" if p < 0.10 else ""

for year in range(2010, 2021):
    # 10-year rolling window: [year-10, year]
    yrs = df.index.get_level_values("DATE").year
    g = df[(yrs >= year - 10) & (yrs <= year)]
    data = g[["exret"] + X_var].dropna()
    exog = sm.add_constant(data[X_var])
    res = FamaMacBeth(data["exret"], exog).fit(cov_type="kernel", kernel="bartlett", bandwidth=12)

    row = {
        "Year": year,
        "Alpha": f"{res.params['const']:.4f}{stars(res.pvalues['const'])}",
        "Adj_R2": round(res.rsquared, 4),
    }
    for v in X_var:
        row[v] = f"{res.params[v]:.4f}{stars(res.pvalues[v])}"

    results.append(row)

results_df = (
    pd.DataFrame(results)
      .set_index("Year")
      .loc[:, ["Alpha"] + X_var + ["Adj_R2"]]
)

display(results_df)


,Alpha,b_mkt,b_smb,b_hml,b_umd,Adj_R2
Year,,,,,,
2010,0.5071***,-0.0179,-0.0468,-0.2479,-0.6999**,0.0003
2011,0.5827***,-0.2330,-0.0727,-0.0143,-0.9415***,0.0006
2012,0.6229***,-0.3591,-0.0913,0.0929,-0.6840***,0.0005
2013,0.5790***,-0.4430*,-0.0184,0.0959,-0.5002***,0.0005
2014,0.6216***,-0.5244**,-0.0355,0.1294,-0.4026***,0.0006
2015,0.5700***,-0.5185*,-0.0414,0.1469,-0.5250***,0.0007
2016,0.6539***,-0.5679**,-0.0882,0.1082,-0.3148*,0.0007
2017,0.6346***,-0.5910**,-0.0334,0.2052,-0.3277*,0.0008
2018,0.6209***,-0.6129**,-0.0728,0.1867,-0.3157*,0.0009
